# OscarNet


## Imports

In [17]:
import keras
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras import layers
from keras import optimizers
from keras import regularizers
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.models import Model
from keras.layers import Dense, Activation,Reshape, Flatten, Conv2D, Dropout, BatchNormalization, Input, MaxPooling2D, Conv2DTranspose
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os, os.path
#from scipy.misc import imread
#from scipy.misc import imshow
import matplotlib
matplotlib.use("Agg")
import cv2

/Users/johnknowles/anaconda/envs/dataweekends/lib/python2.7/site-packages/ipykernel_launcher.py:17: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/johnknowles/anaconda/envs/dataweekends/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/johnknowles/anaconda/envs/dataweekends/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/johnknowles/anaconda/envs/dataweekends/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/johnknowles/anaconda/envs/dataweekends/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in laun

http://cs231n.github.io/transfer-learning/

## Data Processing

rotation_range is a value in degrees (0-180), a range within which to randomly rotate pictures

width_shift and height_shift are ranges (as a fraction of total width or height) within which to randomly translate pictures vertically or horizontally

rescale is a value by which we will multiply the data before any other processing. Our original images consist in RGB coefficients in the 0-255, but such values would be too high for our models to process (given a typical learning rate), so we target values between 0 and 1 instead by scaling with a 1/255. factor.

shear_range is for randomly applying shearing transformations

zoom_range is for randomly zooming inside pictures

horizontal_flip is for randomly flipping half of the images horizontally --relevant when there are no assumptions of horizontal assymetry (e.g. real-world pictures).

fill_mode is the strategy used for filling in newly created pixels, which can appear after a rotation or a width/height shift.

In [18]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)


In [20]:
photo_path = 'dataset_resized/train/'
#categories = ["cardboard","paper", "glass", "metal", "plastic", "trash"]
labels = []
cat = 0

imgs = list()
for label, category in enumerate(categories):
    path = photo_path + category
    alt_path = 'dataset_resized/validation/' + category
    files = os.listdir(path)
    file_count = len(files)
    for i in range(1,file_count):
        print(path)
        #os.rename(path + '//' + category +  str(i) + '.jpg', alt_path + '//' + category +  str((i+1)%file_count) + '.jpg')
        photo =  path + '//' + category +  str(i) + '.jpg'
        img = cv2.imread(photo).astype('float32')/255
        img_scaled = cv2.resize(img, (224, 224))
        img_final = img_scaled.astype('float32')
        imgs.append((img_final, categories.index(category)))
        labels.append(cat)
    cat += 1

dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_resized/train/cardboard
dataset_

KeyboardInterrupt: 

In [ ]:
split = 120
x_train = [l[0] for l in imgs[:split]]
y_train = to_categorical(labels)
x_valid = [l[0] for l in imgs[split:]]
y_valid = [l[1] for l in imgs[split:]]

## ImageNet Network

In [66]:
model = keras.applications.resnet50.ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None)

102604800/102853048 [============================>.] - ETA: 0s

In [22]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='VGGmodel.png')
alt_model = model

In [69]:
alt_model.layers.pop()
alt_model.layers.pop()
alt_model.layers.pop()
model.summary()
model.layers.pop()
#alt_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 230, 230, 3)   0           input_3[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
___________________________________________________________________________________________

In [70]:
alt_model = model
num_layers_retrain = len(model.layers)
num_layers_retrain

176

In [71]:
new_model1 = Sequential() #new model
new_model2 = Sequential() #new model
new_model3 = Sequential() #new model
for layer in alt_model.layers: 
    new_model1.add(layer)
    new_model2.add(layer)
    new_model3.add(layer)
    
    
for layer in alt_model.layers[:num_layers_retrain]:
    layer.trainable = False

ValueError: Input 0 is incompatible with layer res2a_branch1: expected axis -1 of input shape to have value 64 but got shape (None, 55, 55, 256)

## Transfer Network

First: Train a classifier network without retraining any of the layers
            Save these weights
Second: Retrain the weights gradually, slowly 

In [72]:
# how many FC layers and what size 
# what activations should we use
# what is the output size
# 
output = alt_model.layers[len(alt_model.layers)-1].output_shape
output                

(None, 2048)

Network 1

In [73]:
transfer_net1 = Sequential()
transfer_net1.add(Dense(256, input_shape = output,activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros', kernel_regularizer=regularizers.l2(0.05),))
transfer_net1.add(Dropout(0.6))
transfer_net1.add(BatchNormalization())
transfer_net1.add(Dense(7, activation='softmax', kernel_initializer='zeros', bias_initializer='zeros'))
plot_model(transfer_net1, to_file='transfermodel.png')


Network 2

In [44]:
transfer_net2 = Sequential()
transfer_net2.add(Dense(256, input_shape = output,activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
transfer_net2.add(Dropout(0.6))
transfer_net2.add(Dense(128, activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
transfer_net2.add(Dropout(0.4))
transfer_net2.add(Dense(6, activation='softmax', kernel_initializer='zeros', bias_initializer='zeros'))

Network 3

In [45]:
transfer_net3 = Sequential()
transfer_net3.add(Dense(256, input_shape = output,activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
transfer_net3.add(Dropout(0.6))
transfer_net3.add(Dense(256, activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
transfer_net3.add(Dropout(0.6))
transfer_net3.add(Dense(128, activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
transfer_net3.add(Dropout(0.4))
transfer_net3.add(Dense(6, activation='softmax', kernel_initializer='zeros', bias_initializer='zeros'))

In [75]:
new_model1 = model
new_model1.add(transfer_net1)
# new_model2.add(transfer_net2)
# new_model3.add(transfer_net3)

#new_model.summary()

AttributeError: 'list' object has no attribute 'add'

In [47]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_valid = np.array(x_valid)
y_valid = np.array(y_valid)


NameError: name 'y_train' is not defined

# Training Transfer Network

In [32]:
x.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [29]:
batch_size = 32
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2, 
        samplewise_center=False, # set each sample mean to 0
        samplewise_std_normalization=False, # divide each input by its std
        zca_whitening=False, # apply ZCA whitening
        rotation_range=20, # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2, # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2, # randomly shift images vertically (fraction of total height)
        horizontal_flip=True, # randomly flip images
        vertical_flip=False) # randomly flip images

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'dataset_resized/train',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=batch_size,
        save_to_dir='augments/',
        save_format='.png',
        save_prefix = 'augmentTrain',
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels


validation_generator = test_datagen.flow_from_directory(
        'dataset_resized/validation',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')



Found 2498 images belonging to 7 classes.
Found 157 images belonging to 7 classes.


In [30]:
# lr, epsilon, loss, batch_size, 




In [ ]:
models = [new_model1]
batch_sizes = [16, 32, 64]
lrs = [.001, .005, .0001, .0005]
epsilons = [0, .1, .5, 1]
outputs = []



for x in models:
    Wsave = x.get_weights()
    for batch_size in batch_sizes:
        train_generator = train_datagen.flow_from_directory(
            'dataset_resized/train', 
            target_size=(224, 224),  
            batch_size=batch_size,
            class_mode='categorical') 
        for lr in lrs:
            for epsilon in epsilons:
                print("-----------------------")
                print(" ")
                print(batch_size, lr, epsilon)
                adam = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=epsilon, decay=0.0)
                x.set_weights(Wsave)
                x.compile(loss='categorical_crossentropy',
                          optimizer=adam,
                          metrics=['categorical_accuracy'])

                output = x.fit_generator(
                        train_generator,
                        steps_per_epoch=10, 
                        epochs=5,
                        validation_data=validation_generator,
                        validation_steps=30)
                
                print("-----------------------")
                print(" ")
                outputs.append(output)

In [ ]:
print(len(outputs))

## What did we learn?

A larger mini-batch leads to better, faster results (in terms of epochs). However, the smaller batches ran quicker. Across all batches, we saw loss consistently decrease and accuracy consistently increase as the epoch increased, so we decided to balance speed of training with a larger batch size and chose a mini-batch size of 32.

For our lr, we saw that a higher learning rate led to faster, but more variable training. Thus, we decided to 

For our epsilon, we saw that 0 returned nan for loss, and was not a viable option. Upon investgation, we found .5 to be the best parameter for this.

# Plotting Training Results

In [ ]:
plt.style.use("ggplot")
number = 1
N = 5
for H in outputs:
    plt.figure()
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), H.history["categorical_accuracy"], label="train_acc")
    plt.plot(np.arange(0, N), H.history["val_categorical_accuracy"], label="val_acc")
    plt.title("Training Loss and Accuracy on OscarNet" + str(number))
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.savefig("plots/plot_minibatch32" + str(number))
    number += 1


In [34]:
lr = .005
epsilon = .1
batch_size = 32
x = new_model1


adam = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=epsilon, decay=0.0001)

x.compile(loss='categorical_crossentropy',
          optimizer=adam,
          metrics=['categorical_accuracy'])

output = x.fit_generator(
        train_generator,
        steps_per_epoch=15, 
        epochs=5,
        validation_data=validation_generator,
        validation_steps=5)



Epoch 1/5


KeyboardInterrupt: 

In [35]:
x.load_weights('weights/weights_shorttrain1.h5')

In [ ]:
output2 = x.fit_generator(
        train_generator,
        steps_per_epoch=70, 
        epochs=10,
        validation_data=validation_generator,
        validation_steps=5)

Epoch 1/10
70/70 [==============================] - 5561s - loss: 5.5012 - categorical_accuracy: 0.3840 - val_loss: 5.1877 - val_categorical_accuracy: 0.3439
Epoch 2/10
70/70 [==============================] - 4021s - loss: 5.2303 - categorical_accuracy: 0.3951 - val_loss: 5.0049 - val_categorical_accuracy: 0.2866
Epoch 3/10
70/70 [==============================] - 3779s - loss: 5.0755 - categorical_accuracy: 0.3286 - val_loss: 4.9093 - val_categorical_accuracy: 0.4395
Epoch 4/10
70/70 [==============================] - 17535s - loss: 4.8057 - categorical_accuracy: 0.3468 - val_loss: 4.8398 - val_categorical_accuracy: 0.2357
Epoch 5/10
 2/70 [..............................] - ETA: 7380s - loss: 4.7665 - categorical_accuracy: 0.2812 

## Confusion Matrix

In [61]:
test_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = test_datagen.flow_from_directory(
        'dataset_resized/validation',
        target_size=(224, 224),
        batch_size=batch_size,
        shuffle=False,
        class_mode='categorical')

y_true = []
for name in validation_generator.filenames:
    if ("trash" in name): y_true.append(0)
    if ("metal" in name): y_true.append(1)
    if ("plastic" in name): y_true.append(2)
    if ("glass" in name): y_true.append(3)
    if ("paper" in name): y_true.append(4)
    if ("non-waste" in name): y_true.append(5)
    if ("cardboard" in name): y_true.append(6)
y_true = np.array(y_true)

Found 157 images belonging to 7 classes.


In [63]:
probabilities = x.predict_generator(validation_generator, len(validation_generator.filenames))

from sklearn.metrics import confusion_matrix

y_pred = np.array([np.argmax(l) for l in probabilities])

confusion_matrix(y_true, y_pred)

KeyboardInterrupt: 

In [43]:
y_true = np.array([np.argmax(l) for l in probabilities])

In [64]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2, 
        samplewise_center=True, # set each sample mean to 0
        samplewise_std_normalization=True, # divide each input by its std
        zca_whitening=True, # apply ZCA whitening
        rotation_range=20, # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2, # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2, # randomly shift images vertically (fraction of total height)
        horizontal_flip=True, # randomly flip images
        vertical_flip=False) # randomly flip images

train_generator = train_datagen.flow_from_directory(
        'dataset_resized/train',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=batch_size,
        save_to_dir='augments/',
        save_format='.png',
        save_prefix = 'augmentTrain',
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels



Found 2498 images belonging to 7 classes.


In [65]:
output3 = x.fit_generator(
        train_generator,
        steps_per_epoch=70, 
        epochs=10,
        validation_data=validation_generator,
        validation_steps=5)

Epoch 1/10


/Users/johnknowles/anaconda/envs/dataweekends/lib/python2.7/site-packages/keras/preprocessing/image.py:524: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


 6/70 [=>............................] - ETA: 3544s - loss: 20.9845 - categorical_accuracy: 0.2448

KeyboardInterrupt: 